# TRAINING MODEL (CNN+RNN)

#IMPORTING REQUIRED LIBRARIES AND MODULES

In [ ]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
import pickle

# CONNECTING WITH GOOGLE DRIVE


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#IMPORTING INTENT DATASET IN JSON FORMAT

In [ ]:
with open('/content/gdrive/MyDrive/SoftComputing/intents.json') as file:
    data = json.load(file)

#FEATURE ENGINEERING

In [ ]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [ ]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])


In [ ]:
num_classes = len(labels)

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"


In [ ]:
# adding out of vocabulary token
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) 
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)


#Model Building ( CNN Sequential + RNN )

In [ ]:
# CNN Sequential model building
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
# Compliling everything
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# displaying model architecture/design
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 37)                629       
                                                                 
Total params: 17,173
Trainable params: 17,173
Non-trainable params: 0
____________________________________________________

In [ ]:
# defining epochs as 550
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)
model.compile(loss='mse', optimizer='adam')

Epoch 1/550
7/7 [==============================] - 0s 2ms/step - loss: 3.6107 - accuracy: 0.0207
Epoch 2/550
7/7 [==============================] - 0s 2ms/step - loss: 3.6069 - accuracy: 0.0415
Epoch 3/550
7/7 [==============================] - 0s 2ms/step - loss: 3.6036 - accuracy: 0.0466
Epoch 4/550
7/7 [==============================] - 0s 2ms/step - loss: 3.6002 - accuracy: 0.0415
Epoch 5/550
7/7 [==============================] - 0s 2ms/step - loss: 3.5962 - accuracy: 0.0415
Epoch 6/550
7/7 [==============================] - 0s 2ms/step - loss: 3.5922 - accuracy: 0.0415
Epoch 7/550
7/7 [==============================] - 0s 2ms/step - loss: 3.5875 - accuracy: 0.0415
Epoch 8/550
7/7 [==============================] - 0s 2ms/step - loss: 3.5819 - accuracy: 0.0415
Epoch 9/550
7/7 [==============================] - 0s 4ms/step - loss: 3.5751 - accuracy: 0.0415
Epoch 10/550
7/7 [==============================] - 0s 2ms/step - loss: 3.5671 - accuracy: 0.0415
Epoch 11/550
7/7 [===========

In [ ]:
# saving model
model.save('/content/gdrive/MyDrive/SoftComputing/chat_model')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/SoftComputing/chat_model/assets


In [ ]:
# saving tokenizer
with open('/content/gdrive/MyDrive/SoftComputing/tokenizer.pickle' ,'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# saving label encoder
with open('/content/gdrive/MyDrive/SoftComputing/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)